In [1]:
import numpy as np
from qiskit_experiments.library import StandardRB, InterleavedRB
from qiskit_experiments.framework import ParallelExperiment, BatchExperiment
import qiskit.circuit.library as circuits

In [2]:
from qiskit import assemble
from qiskit_ibm_provider import IBMProvider
#from qiskit import IBMQ
from qiskit import execute
from qiskit.tools import job_monitor

In [3]:
IBMProvider.save_account('', overwrite=True)
IBMProvider()

<IBMProvider>

In [4]:
provider = IBMProvider(instance="ibm-q-ncsu/nc-state/vlsi-cad-tools")
backend = provider.get_backend('ibm_hanoi')

In [5]:
lengths_x_qubit = np.arange(1, 10, 3)
lengths_1_qubit = np.arange(1, 800, 200)
num_samples = 10
seed = 1010
qubits = (0, 1,2,3,4,5)

# Run a 1-qubit RB experiment on qubits 1 to determine the error-per-gate of 1-qubit gates
single_exps = BatchExperiment(
    [
        StandardRB([qubit], lengths_1_qubit, num_samples=num_samples, seed=seed)
        for qubit in qubits
    ],
    flatten_results=True,
)
expdata_1q = single_exps.run(backend).block_for_results()

In [6]:
exp_xq = StandardRB(qubits, lengths_x_qubit, num_samples=num_samples, seed=seed)

# Use the EPG data of the 1-qubit runs to ensure correct x-qubit EPG computation
exp_xq.analysis.set_options(epg_1_qubit=expdata_1q.analysis_results())

# Run the x-qubit experiment
expdata_xq = exp_xq.run(backend, shots = 4000).block_for_results()

# View result data
print("Gate error ratio: %s" % expdata_xq.experiment.analysis.options.gate_error_ratio)
display(expdata_xq.figure(0))
for result in expdata_xq.analysis_results():
    print(result)

Job was cancelled before completion [Job ID: cpjs54w6q5h0008bjmb0]


Gate error ratio: default


ExperimentEntryNotFound: 'Figure 0 not found.'

In [ ]:
exp_xq.circuits()[29].decompose().depth()

In [ ]:
exp_xq.circuits()[0].draw('mpl')

In [ ]:
print(c.decompose().depth())
c.decompose().draw("mpl")